In [1]:
import tensorflow as tf
import numpy as np
import nengo_dl
import nengo_loihi
import nengo

# TF Model

In [2]:
tf.random.set_seed(0)
channels = "channels_last"
inp = tf.keras.Input((32, 32, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), data_format=channels, activation="relu", use_bias=False)(inp)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(32, (3, 3), data_format=channels, activation="relu", use_bias=False)(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.MaxPool2D((2, 2), data_format=channels)(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Conv2D(64, (3, 3), data_format=channels, activation="relu", use_bias=False)(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), data_format=channels, activation="relu", use_bias=False)(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.MaxPool2D((2, 2), data_format=channels)(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Conv2D(128, (3, 3), data_format=channels, activation="relu", use_bias=False)(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(128, (3, 3), data_format=channels, activation="relu", use_bias=False)(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
# x = tf.keras.layers.MaxPool2D((2, 2), data_format="channels_last")(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(10)(x)

model = tf.keras.Model(inputs=inp, outputs=x)

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        864       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18432     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36864 

In [4]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
train_x = train_x.astype(np.float32) / 127.5 - 1
test_x = test_x.astype(np.float32) / 127.5 - 1

train_y = np.eye(10, dtype=np.float32)[train_y].squeeze(axis=1)
test_y = np.eye(10, dtype=np.float32)[test_y].squeeze(axis=1)

# train_x, test_x = np.moveaxis(train_x, -1, 1), np.moveaxis(test_x, -1, 1) # Make Channel's Last

print(train_x.shape, test_x.shape)
# Flatten the images.
train_x = train_x.reshape(train_x.shape[0], 1, -1)
test_x = test_x.reshape(test_x.shape[0], 1, -1)
print(train_x.shape, test_x.shape)
train_y = train_y.reshape(train_y.shape[0], 1, -1)
print(train_y.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 1, 3072) (10000, 1, 3072)
(50000, 1, 10)


# Training NengoDL Model

In [6]:
np.random.seed(0)
ndl_model = nengo_dl.Converter(model, swap_activations=
                               {
                                 tf.keras.activations.relu: nengo_loihi.neurons.LoihiSpikingRectifiedLinear()
                               },
                                scale_firing_rates=400
                              )
ndl_input = ndl_model.inputs[inp]
ndl_output = ndl_model.outputs[x]

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")


In [7]:
with nengo_dl.Simulator(ndl_model.net, minibatch_size=100, seed=0, progress_bar=False) as ndl_sim:
  losses  = {
    ndl_output: tf.losses.CategoricalCrossentropy(from_logits=True, )
  }
  ndl_sim.compile(
    optimizer=tf.optimizers.Adam(learning_rate=1e-3, decay=1e-5),
    loss=losses,
    metrics=["accuracy"]
  )
  ndl_sim.fit(
    {ndl_input: train_x, 
#      "n_steps": np.ones((train_x.shape[0], 1), dtype=np.int32), 
#      "dense_1.0.bias": np.ones((train_x.shape[0], 10, 1), dtype=np.int32)
    },
    {ndl_output: train_y},
    epochs=128,
  )
  ndl_sim.save_params("./ndl_trained_params_sfr_400_epochs_128")

Epoch 1/128
500/500 [==============================] - 9s 17ms/step - loss: 1.8451 - probe_loss: 1.8451 - probe_accuracy: 0.3006
Epoch 2/128
500/500 [==============================] - 8s 17ms/step - loss: 1.4390 - probe_loss: 1.4390 - probe_accuracy: 0.4819
Epoch 3/128
500/500 [==============================] - 8s 17ms/step - loss: 1.2405 - probe_loss: 1.2405 - probe_accuracy: 0.5649
Epoch 4/128
500/500 [==============================] - 9s 17ms/step - loss: 1.1188 - probe_loss: 1.1188 - probe_accuracy: 0.6115
Epoch 5/128
500/500 [==============================] - 9s 17ms/step - loss: 1.0307 - probe_loss: 1.0307 - probe_accuracy: 0.6424
Epoch 6/128
500/500 [==============================] - 8s 17ms/step - loss: 0.9651 - probe_loss: 0.9651 - probe_accuracy: 0.6697
Epoch 7/128
500/500 [==============================] - 8s 17ms/step - loss: 0.9055 - probe_loss: 0.9055 - probe_accuracy: 0.6913
Epoch 8/128
500/500 [==============================] - 8s 17ms/step - loss: 0.8696 - probe_loss: 

500/500 [==============================] - 9s 17ms/step - loss: 0.4926 - probe_loss: 0.4926 - probe_accuracy: 0.8382
Epoch 65/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4858 - probe_loss: 0.4858 - probe_accuracy: 0.8393
Epoch 66/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4806 - probe_loss: 0.4806 - probe_accuracy: 0.8400
Epoch 67/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4943 - probe_loss: 0.4943 - probe_accuracy: 0.8373
Epoch 68/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4774 - probe_loss: 0.4774 - probe_accuracy: 0.8409
Epoch 69/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4823 - probe_loss: 0.4823 - probe_accuracy: 0.8412
Epoch 70/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4795 - probe_loss: 0.4795 - probe_accuracy: 0.8417
Epoch 71/128
500/500 [==============================] - 9s 17ms/step - loss: 0.4773 - probe_loss: 0.477

# Testing NengoDL Model

In [9]:
layers = [l for l in model.layers]
layer_objs_lst = []
x0 = layers[0].output
layer_objs_lst.append(x0)
for i in range(1, len(layers)):
  if layers[i].name.startswith("dropout"):
    continue
  x0 = layers[i](x0)
  layer_objs_lst.append(x0)

new_model = tf.keras.Model(inputs=layers[0].input, outputs=x0)
new_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        864       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18432     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        36864     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0   

In [6]:
test_x_tiled = np.tile(test_x, (1, 120, 1))
ndl_model_test = nengo_dl.Converter(
    new_model,
    scale_firing_rates=400,
    synapse=0.005,
    swap_activations={tf.keras.activations.relu: nengo_loihi.neurons.LoihiSpikingRectifiedLinear()},
    )
ndl_input = ndl_model_test.inputs[inp]
ndl_output = ndl_model_test.outputs[x0]

/home/rgaurav/miniconda3/envs/nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")


In [7]:
#for i, layer in enumerate(new_model.layers):
for i, layer in enumerate(layer_objs_lst):
    print(i, layer)

0 Tensor("input_1:0", shape=(None, 32, 32, 3), dtype=float32)
1 Tensor("conv2d_6/Identity:0", shape=(None, 30, 30, 32), dtype=float32)
2 Tensor("conv2d_1_1/Identity:0", shape=(None, 28, 28, 32), dtype=float32)
3 Tensor("max_pooling2d_2/Identity:0", shape=(None, 14, 14, 32), dtype=float32)
4 Tensor("conv2d_2_1/Identity:0", shape=(None, 12, 12, 64), dtype=float32)
5 Tensor("conv2d_3_1/Identity:0", shape=(None, 10, 10, 64), dtype=float32)
6 Tensor("max_pooling2d_1_1/Identity:0", shape=(None, 5, 5, 64), dtype=float32)
7 Tensor("conv2d_4_1/Identity:0", shape=(None, 3, 3, 128), dtype=float32)
8 Tensor("conv2d_5_1/Identity:0", shape=(None, 1, 1, 128), dtype=float32)
9 Tensor("flatten_1/Identity:0", shape=(None, 128), dtype=float32)
10 Tensor("dense_2/Identity:0", shape=(None, 128), dtype=float32)
11 Tensor("dense_1_1/Identity:0", shape=(None, 10), dtype=float32)


In [8]:
ndl_model_test.layers[layer_objs_lst[3]].probeable

('output',)

In [9]:
# with ndl_model_test.net:
#     mp1_probe = nengo.Probe(ndl_model_test.layers[layer_objs_lst[3]])
#     mp2_probe = nengo.Probe(ndl_model_test.layers[layer_objs_lst[6]])

In [9]:
with nengo_dl.Simulator(
    ndl_model_test.net, minibatch_size=100, progress_bar=False) as ndl_sim_test:
  ndl_sim_test.load_params("./ndl_trained_params_sfr_400_epochs_128")#_epochs_64")
  sim_data = ndl_sim_test.predict({ndl_input: test_x_tiled[:]})

In [10]:
pred_clss = np.argmax(sim_data[ndl_output][:, -1, :], axis=-1)

In [11]:
100*np.mean(pred_clss == np.argmax(test_y[:], axis=-1))

81.05

In [12]:
sim_data[ndl_output].shape
np.save("NDL_pred_output_epochs_128_sfr_400", sim_data[ndl_output])
# np.save("NDL_pred_output_epochs_128_sfr_400_mp1_actvns", sim_data[mp1_probe])
# np.save("NDL_pred_output_epochs_128_sfr_400_mp2_actvns", sim_data[mp2_probe])

In [ ]:
for lyr in ndl_model_test.net.layers:
  print(lyr)

In [ ]:
for lyr in ndl_model_test.net.layers:
  print(lyr)